## Libraries

In [3]:
# !pip install yfinance

In [4]:
import yfinance as yf
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
# import plotly
# import plotly.express as px
# import plotly.graph_objects as go
# pd.options.plotting.backend = "plotly"
import numpy as np
import boto3
from io import StringIO
from sagemaker import get_execution_role
import warnings
warnings.filterwarnings("ignore")
import pickle

## Functions

In [5]:
# Function to save a .csv in S3 from a DataFrame of our SageMaker python notebook
def _write_dataframe_to_csv_on_s3(DESTINATION, dataframe, filename):
    """ Write a dataframe to a CSV on S3 """
    print(
        "Writing {} records to '{}' in bucket '{}'.".format(
            len(dataframe), filename, DESTINATION
        )
    )
    # Create buffer
    csv_buffer = StringIO()
    # Write dataframe to buffer
    dataframe.to_csv(csv_buffer, sep=",", index=True)
    # Create S3 object
    s3_resource = boto3.resource("s3")
    # Write buffer to S3 object
    s3_resource.Object(DESTINATION, filename).put(Body=csv_buffer.getvalue())

## How to use yfinance library

### Get ticker info

In [6]:
mystock = "^N100" # ticker code per Yahoo Finance
myticker = yf.Ticker(mystock)

In [7]:
print(myticker.info)

{'previousClose': 1235.02, 'regularMarketOpen': 1237.76, 'twoHundredDayAverage': 1119.757, 'trailingAnnualDividendYield': None, 'payoutRatio': None, 'volume24Hr': None, 'regularMarketDayHigh': 1238.1, 'navPrice': None, 'averageDailyVolume10Day': 181605828, 'totalAssets': None, 'regularMarketPreviousClose': 1235.02, 'fiftyDayAverage': 1207.1918, 'trailingAnnualDividendRate': None, 'open': 1237.76, 'toCurrency': None, 'averageVolume10days': 181605828, 'expireDate': None, 'yield': None, 'algorithm': None, 'dividendRate': None, 'exDividendDate': None, 'beta': None, 'circulatingSupply': None, 'startDate': None, 'regularMarketDayLow': 1228.38, 'priceHint': 2, 'currency': 'EUR', 'regularMarketVolume': 0, 'lastMarket': None, 'maxSupply': None, 'openInterest': None, 'marketCap': None, 'volumeAllCurrencies': None, 'strikePrice': None, 'averageVolume': 213927488, 'priceToSalesTrailing12Months': None, 'dayLow': 1228.38, 'ask': 0, 'ytdReturn': None, 'askSize': 0, 'volume': 0, 'fiftyTwoWeekHigh': 12

In [8]:
# for key in myticker.info:
#     print(key)

# print(myticker.info["longName"])
# print(myticker.info["sector"])
# print(myticker.info["industry"])

In [9]:
# mytickers = yf.Tickers(["MC.PA", "RDSA.AS"])

In [10]:
# for ticker in mytickers.tickers:
#     print(mytickers.tickers[ticker].info["longName"])

### Parameters for the history function

### Get history of a stock

In [11]:
# hist = myticker.history(period="1y") # check docs for parameters

In [12]:
# hist.head()

In [13]:
# hist["Close"].plot(figsize=(16,9), grid=True, title=f"{mystock}\n{myticker.info['longName']}") # without Plotly backend

### yf.download function

In [14]:
# Stocks comprising the euronext 100
euronext100 = ['KBC.BR', 'JMT.LS', 'BNP.PA', 'BN.PA', 'GBLB.BR', 'KN.PA', 'AGS.BR', 'LR.PA', 'MC.PA', 'ABI.BR', 'EDEN.PA', 'RDSA.AS', 'OR.PA', 'CAP.PA', 'SAN.PA', 'GALP.LS', 'SW.PA', 'INGA.AS', 'KER.PA', 'PHIA.AS', 'ATO.PA', 'ILD.PA', 'COV.PA', 'AIR.PA', 'AC.PA', 'TEP.PA', 'UBI.PA', 'ASML.AS', 'RAND.AS', 'VIE.PA']

# All stocks on Euronext Paris, Amsterdam & Brussels
euronext_pab = ['2CRSI.PA', '2MX.PA', 'ASP.PA', 'AALB.AS', 'ABI.BR', 'AB.PA', 'ABCA.PA', 'ABEO.PA', 'ABNX.PA', 'ABVX.PA', 'ABN.AS', 'ABO.BR', 'ACPH.BR', 'ACAN.PA', 'ACCEL.AS', 'ACCB.BR', 'AC.PA', 'ACNV.PA', 'AXS.AS', 'ACKB.BR', 'EOS.PA', 'ATI.PA', 'ALDV.PA', 'ALP.PA', 'ADOC.PA', 'ADP.PA', 'ADUX.PA', 'ADV.PA', 'ADVIC.PA', 'ADVI.PA', 'ADYEN.AS', 'AED.BR', 'AGN.AS', 'AKOM.PA', 'AGS.BR', 'AGFB.BR', 'AD.AS', 'AF.PA', 'AI.PA', 'AIR.PA', 'AJAX.AS', 'AKA.PA', 'AKW.PA', 'AKZA.AS', 'AAA.PA', 'ABIO.PA', 'ALD.PA', 'ALFEN.AS', 'CDANV.PA', 'CDA.PA', 'ALM.PA', 'ALO.PA', 'ALONV.PA', 'LTA.PA', 'ALTA.PA', 'AREIT.PA', 'ATE.PA', 'ALTUR.PA', 'ALX.AS', 'AMG.AS', 'AMPLI.PA', 'ACOMO.AS', 'AMUN.PA', 'APAM.AS', 'APM.PA', 'ARCAD.AS', 'MT.AS', 'ARG.PA', 'ARGX.BR', 'AKE.PA', 'ARTE.PA', 'PRC.PA', 'ARTO.PA', 'ASC.BR', 'ASIT.BR', 'ASM.AS', 'ASML.AS', 'ASRNL.AS', 'ASY.PA', 'ATA.PA', 'ATEME.PA', 'ATEB.BR', 'ATO.PA', 'ATRS.AS', 'AUB.PA', 'AUGR.PA', 'AURE.PA', 'AURS.PA', 'AVTX.AS', 'AVT.PA', 'CS.PA', 'CSNV.PA', 'AXW.PA', 'BSGR.AS', 'BCRA.PA', 'BAIN.PA', 'BALTA.BR', 'BALYO.PA', 'BAMNB.AS', 'BANI.BR', 'BUI.PA', 'BAR.BR', 'BFIT.AS', 'BASS.PA', 'BLC.PA', 'BESI.AS', 'BEFB.BR', 'BEKB.BR', 'FBEL.PA', 'BELR.BR', 'BELU.BR', 'BEN.PA', 'BBED.AS', 'BEVER.AS', 'BB.PA', 'BIG.PA', 'BCART.BR', 'BIM.PA', 'BLEE.PA', 'BNP.PA', 'BOI.PA', 'BOL.PA', 'BOLNV.PA', 'BON.PA', 'BOTHE.BR', 'BOOST.PA', 'BOKA.AS', 'BSD.PA', 'BGHL.AS', 'BGHS.AS', 'EN.PA', 'ENNV.PA', 'BPOST.BR', 'BNB.BR', 'BREB.BR', 'BRILL.AS', 'BRNL.AS', 'BVI.PA', 'BUR.PA', 'CAT31.PA', 'CAFO.PA', 'CBDG.PA', 'CAMB.BR', 'CAPLI.PA', 'CAP.PA', 'CPINV.BR', 'CARM.PA', 'CARP.PA', 'CA.PA', 'CO.PA', 'CAS.PA', 'CATG.PA', 'CTRG.PA', 'CATR.PA', 'CBOT.PA', 'CGM.PA', 'CGR.PA', 'CYAD.BR', 'CENER.BR', 'CFEB.BR', 'CFI.PA', 'CGG.PA', 'CRI.PA', 'CHSR.PA', 'CDI.PA', 'CIB.PA', 'COMB.BR', 'CLA.PA', 'CMCOM.AS', 'COM.PA', 'CNV.PA', 'CNP.PA', 'CCEP.AS', 'COFA.PA', 'COFB.BR', 'COH.PA', 'RE.PA', 'COLR.BR', 'CRBN.AS', 'CLB.AS', 'COUR.PA', 'COV.PA', 'COVH.PA', 'CRAP.PA', 'CRAV.PA', 'CRBP2.PA', 'CIV.PA', 'CRLA.PA', 'CRLO.PA', 'CMO.PA', 'CNF.PA', 'CCN.PA', 'CAF.PA', 'CRSU.PA', 'CRTO.PA', 'ACA.PA', 'OPTI.BR', 'CROS.PA', 'SX.PA', 'CTAC.AS', 'CTPNV.AS', 'CLEX.BR', 'DIE.BR', 'BN.PA', 'AM.PA', 'DSYNV.PA', 'DSY.PA', 'DBV.PA', 'DECB.BR', 'DLTA.PA', 'DBG.PA', 'DVT.PA', 'DGB.AS', 'DGE.PA', 'DGM.PA', 'DPAM.PA', 'DPA.AS', 'DSC2.AS', 'DSC2S.AS', 'DSM.AS', 'EAS2P.AS', 'ECONB.BR', 'EDEN.PA', 'EDF.PA', 'EFIC1.AS', 'EFICT.AS', 'GID.PA', 'FGR.PA', 'EKI.PA', 'EKOP.BR', 'ELEC.PA', 'EEM.PA', 'ELI.BR', 'ELIOR.PA', 'ELIS.PA', 'ENGI.PA', 'EPS.PA', 'ENVI.AS', 'EOSI.PA', 'ERA.PA', 'ERYP.PA', 'ESG.AS', 'ESGT.AS', 'ESI.PA', 'ESP.AS', 'ELNV.PA', 'EL.PA', 'ES.PA', 'EFI.PA', 'RF.PA', 'EUR.PA', 'ECT.AS', 'ECMPA.AS', 'ERFNV.PA', 'ERF.PA', 'EURN.BR', 'ENX.PA', 'ECP.PA', 'EUCAR.PA', 'ETL.PA', 'EGR.PA', 'EVS.BR', 'EXAC.PA', 'EXE.PA', 'EXM.BR', 'EPCP.PA', 'EXPL.PA', 'FAGR.BR', 'FAST.AS', 'EO.PA', 'FAUV.PA', 'FAYE.PA', 'FDJ.PA', 'FCMC.PA', 'FALG.PA', 'SACI.PA', 'ORIA.PA', 'FGA.PA', 'BERR.PA', 'EBPF.PA', 'FOAF.PA', 'FINM.PA', 'ODET.PA', 'FNTS.PA', 'FIPP.PA', 'FLOB.BR', 'FLOW.AS', 'FLUX.BR', 'FNAC.PA', 'FNG.BR', 'FPN.PA', 'LEBL.PA', 'FATL.PA', 'EURS.PA', 'INEA.PA', 'FLY.PA', 'SPEL.PA', 'FORE.PA', 'FFARM.AS', 'FOU.BR', 'LFDE.PA', 'FREY.PA', 'FUR.AS', 'GLPG.AS', 'GALIM.PA', 'GAM.PA', 'GBLB.BR', 'GEA.PA', 'GFC.PA', 'GNE.PA', 'GENX.PA', 'GNRO.PA', 'GNFT.PA', 'GKTX.PA', 'GV.PA', 'SIGHT.PA', 'GOJXN.AS', 'GET.PA', 'GIMB.BR', 'GLO.PA', 'GLOG.BR', 'GPE.PA', 'GRVO.PA', 'GVNV.AS', 'GREEN.BR', 'CEN.PA', 'FLO.PA', 'GOE.PA', 'IRD.PA', 'GJAJ.PA', 'OPN.PA', 'PARP.PA', 'SFPI.PA', 'GTBP.PA', 'GTT.PA', 'GBT.PA', 'GBTNV.PA', 'GUI.PA', 'HAL.AS', 'HAMO.BR', 'PIG.PA', 'HEIJM.AS', 'HEIA.AS', 'HEIO.AS', 'RMS.PA', 'HEXA.PA', 'HF.PA', 'HCO.PA', 'HIPAY.PA', 'HOLCO.AS', 'HOMI.BR', 'HOP.PA', 'HDP.PA', 'HDG.AS', 'HUNDP.AS', 'HYDRA.AS', 'HYL.BR', 'IBAB.BR', 'ICAD.PA', 'ICT.AS', 'IDL.PA', 'IDIP.PA', 'IEP.BR', 'IEX.AS', 'IGE.PA', 'ILD.PA', 'IMCD.AS', 'NK.PA', 'IMMOU.BR', 'IMDA.PA', 'IMMO.BR', 'INCLU.BR', 'INFE.PA', 'INF.PA', 'INGA.AS', 'IPH.PA', 'INPST.AS', 'ITP.PA', 'INTER.AS', 'INTO.BR', 'ITXT.PA', 'IVA.PA', 'IPN.PA', 'IPS.PA', 'IPSNV.PA', 'ITE.PA', 'JBOG.PA', 'JCQ.PA', 'DEC.PA', 'JDEP.AS', 'JEN.BR', 'TKWY.AS', 'KOF.PA', 'KBC.BR', 'KBCA.BR', 'KENDR.AS', 'KER.PA', 'KEYW.BR', 'KDS.AS', 'KIN.BR', 'LI.PA', 'KORI.PA', 'KPN.AS', 'OR.PA', 'ORNV.PA', 'LACR.PA', 'LHN.PA', 'MMB.PA', 'LAT.PA', 'LPE.PA', 'LVIDE.AS', 'LOUP.PA', 'LEAS.BR', 'LBON.PA', 'LSS.PA', 'LSSNV.PA', 'LR.PA', 'LIN.PA', 'FII.PA', 'LNA.PA', 'LOTB.BR', 'BOLS.AS', 'LBIRD.PA', 'MC.PA', 'LYS.PA', 'MRM.PA', 'MDM.PA', 'MALT.PA', 'MTU.PA', 'MAN.PA', 'MAREL.AS', 'IAM.PA', 'MKEA.PA', 'MAU.PA', 'MBWS.PA', 'MCPHY.PA', 'MDXH.BR', 'EDI.PA', 'MEDCL.PA', 'MELE.BR', 'MEMS.PA', 'MERY.PA', 'MRK.PA', 'MRN.PA', 'MRNNV.PA', 'METEX.PA', 'MMT.PA', 'ML.PA', 'MLNV.PA', 'MUN.PA', 'MIKO.BR', 'MITRA.BR', 'NEDSE.AS', 'MONC.PA', 'MONT.BR', 'MOP.BR', 'MOPF.BR', 'MORE.AS', 'MOUR.BR', 'GREV.PA', 'NACON.PA', 'NANO.PA', 'KN.PA', 'NAVYA.PA', 'NEDAP.AS', 'NEOEN.PA', 'NRP.AS', 'NRO.PA', 'NSE.AS', 'NEWAY.AS', 'NEX.PA', 'NXI.PA', 'NEXTS.PA', 'COX.PA', 'NN.AS', 'NOKIA.PA', 'NOVI.AS', 'NR21.PA', 'NRG.PA', 'NSI.AS', 'NYR.BR', 'NYXH.BR', 'OCI.AS', 'SBT.PA', 'OLG.PA', 'ONTEX.BR', 'ORA.PA', 'OBEL.BR', 'ORANW.AS', 'ORAP.PA', 'KAZI.PA', 'ORDI.AS', 'OREGE.PA', 'ORP.PA', 'OSE.PA', 'OXUR.BR', 'PID.PA', 'PAR.PA', 'PARRO.PA', 'PSAT.PA', 'PAT.PA', 'PAY.BR', 'PCA.PA', 'RI.PA', 'PERR.PA', 'PSH.AS', 'PEUG.PA', 'PHA.PA', 'PHARM.AS', 'INPHI.AS', 'PHIA.AS', 'PIC.BR', 'VAC.PA', 'PVL.PA', 'POM.PA', 'PORF.AS', 'PNL.AS', 'POXEL.PA', 'PPLAA.AS', 'PPLAB.AS', 'PPLA.AS', 'PREC.PA', 'PROAC.PA', 'PWG.PA', 'PROL.PA', 'PRX.AS', 'PROX.BR', 'PSB.PA', 'PUB.PA', 'QRF.BR', 'QDT.PA', 'QFG.BR', 'RAL.PA', 'GDS.PA', 'RAND.AS', 'REC.BR', 'RX.PA', 'REINA.AS', 'REN.AS', 'RCO.PA', 'RNO.PA', 'RWI.AS', 'RES.BR', 'RET.BR', 'RXL.PA', 'RBT.PA', 'CBR.PA', 'CBE.PA', 'RBO.PA', 'ROOD.AS', 'ENGB.BR', 'ROTH.PA', 'ROU.BR', 'RDSA.AS', 'RDSB.AS', 'RUI.PA', 'SK.PA', 'SAF.PA', 'SGO.PA', 'SGONV.PA', 'SABE.PA', 'SAMS.PA', 'SAN.PA', 'SANNV.PA', 'DIM.PA', 'SAVE.PA', 'SBMO.AS', 'CBSM.PA', 'SCHD.BR', 'SLB.PA', 'SU.PA', 'SCR.PA', 'SCHP.PA', 'SGRO.PA', 'SLCO.PA', 'SELER.PA', 'SEQUA.BR', 'SEFER.PA', 'SESG.PA', 'SESL.PA', 'SRP.PA', 'SHUR.BR', 'SIFG.AS', 'GIRO.PA', 'LIGHT.AS', 'SII.PA', 'SIOE.BR', 'SIP.BR', 'SLIGR.AS', 'SMAR.BR', 'SMCP.PA', 'SMTPC.PA', 'SNOW.AS', 'SFCA.PA', 'GLE.PA', 'SW.PA', 'SEC.PA', 'SOF.BR', 'SOG.PA', 'SOI.PA', 'LOCAL.PA', 'S30.PA', 'SOLV.BR', 'SOLB.BR', 'SO.PA', 'SOP.PA', 'SPA.BR', 'SPIE.PA', 'SQI.PA', 'DPT.PA', 'STF.PA', 'STLA.PA', 'STRN.AS', 'STM.PA', 'SEV.PA', 'SWP.PA', 'SDG.PA', 'TKTT.PA', 'TAYN.PA', 'TCH.PA', 'TE.PA', 'FTI.PA', 'TNET.BR', 'TEP.PA', 'TVRB.PA', 'TERB.BR', 'TESB.BR', 'TES.PA', 'TFG.AS', 'TEXF.BR', 'TFI.PA', 'TFF.PA', 'HO.PA', 'HONV.PA', 'THEP.PA', 'TBIRD.AS', 'TIE.AS', 'TKO.PA', 'TINC.BR', 'TIPI.PA', 'TITC.BR', 'TWEKA.AS', 'TOM2.AS', 'FP.PA', 'EC.PA', 'TOUP.PA', 'EIFF.PA', 'TNG.PA', 'TRI.PA', 'TUB.BR', 'UBI.PA', 'UCB.BR', 'UMI.BR', 'URW.AS', 'UNBL.PA', 'UPG.BR', 'UNA.AS', 'UFF.PA', 'FPG.PA', 'VLK.AS', 'FR.PA', 'VK.PA', 'VLA.PA', 'VALUE.AS', 'PREVA.AS', 'VAN.BR', 'VASTN.AS', 'VASTB.BR', 'VIE.PA', 'VEON.AS', 'VRLA.PA', 'VMX.PA', 'VETO.PA', 'VGP.BR', 'VCT.PA', 'VDLO.PA', 'VIL.PA', 'RIN.PA', 'DG.PA', 'VIO.BR', 'VIRP.PA', 'SDT.PA', 'VIV.PA', 'VIVNV.PA', 'VVY.AS', 'VTA.AS', 'VLTSA.PA', 'VPK.AS', 'VRAP.PA', 'WEB.BR', 'WAVE.PA', 'WDP.BR', 'MF.PA', 'WHA.AS', 'WEHB.BR', 'WKL.AS', 'WLN.PA', 'XFAB.PA', 'XIL.PA', 'XIOR.BR', 'XPO.PA', 'YATRA.AS', 'CV.PA', 'ZENT.BR', '^FCHI', '^N100']

# Selection of stocks & reference indices for testing
stocks_for_testing = ["EDF.PA", "ENGI.PA", "ABIO.PA", "ELEC.PA", "NEOEN.PA", "NSE.AS", "VLTSA.PA", "^FCHI", "^N100"]

# mystocks = euronext_pab
mystocks = euronext_pab

In [15]:
# ?yf.download

In [16]:
# Pull closing prices and volumes. Check docs for all parameters
data = yf.download(
    tickers=mystocks,
    period="2y",
    interval="1d",
    threads=True,
)[["Adj Close", "Volume"]]

[*********************100%***********************]  706 of 706 completed

12 Failed downloads:
- LSSNV.PA: No data found, symbol may be delisted
- CV.PA: No data found, symbol may be delisted
- DSYNV.PA: No data found, symbol may be delisted
- CDANV.PA: No data found, symbol may be delisted
- ESGT.AS: No data found, symbol may be delisted
- BOLNV.PA: No data found, symbol may be delisted
- VIVNV.PA: No data found, symbol may be delisted
- DSC2S.AS: No data found, symbol may be delisted
- SGONV.PA: No data found for this date range, symbol may be delisted
- ELNV.PA: No data found, symbol may be delisted
- EFICT.AS: No data found, symbol may be delisted
- ORNV.PA: No data found, symbol may be delisted


In [17]:
# Store volumes in separate dataframe
df_volumes = data["Volume"]
df_volumes = df_volumes[mystocks]

In [18]:
df_volumes.shape

(514, 706)

In [19]:
df_volumes.head()

,2CRSI.PA,2MX.PA,ASP.PA,AALB.AS,ABI.BR,AB.PA,ABCA.PA,ABEO.PA,ABNX.PA,ABVX.PA,...,WLN.PA,XFAB.PA,XIL.PA,XIOR.BR,XPO.PA,YATRA.AS,CV.PA,ZENT.BR,^FCHI,^N100
Date,,,,,,,,,,,,,,,,,,,,,
2019-04-30,NaN,NaN,NaN,364254.0,NaN,NaN,27609,NaN,35523,NaN,...,NaN,NaN,NaN,NaN,222,1705.0,NaN,NaN,NaN,195737100.0
2019-05-02,3440.0,NaN,14233.0,353729.0,1837150.0,58929.0,34768,1502.0,1942832,26279.0,...,1374143.0,83450.0,3954.0,12918.0,52,0.0,NaN,355.0,92443600.0,189853200.0
2019-05-03,10453.0,NaN,26841.0,186033.0,1360927.0,67989.0,39904,7117.0,122584,11171.0,...,3232501.0,33513.0,5270.0,22579.0,23,0.0,NaN,64.0,67538000.0,139657900.0
2019-05-06,6049.0,NaN,39242.0,300758.0,1509396.0,61528.0,79039,3248.0,351441,19389.0,...,683904.0,57915.0,6678.0,19486.0,0,0.0,NaN,413.0,77234200.0,143696000.0
2019-05-07,12945.0,NaN,44480.0,415051.0,2471377.0,41638.0,24800,239.0,106026,3434.0,...,983411.0,84126.0,4518.0,7836.0,15,200.0,NaN,1057.0,93789000.0,204632800.0


In [20]:
# Save df_prices to s3
_write_dataframe_to_csv_on_s3("euronext-stocks", df_volumes, "df_volumes.csv")

Writing 514 records to 'df_volumes.csv' in bucket 'euronext-stocks'.


In [21]:
# Store prices in separate dataframe
df_prices = data["Adj Close"]
df_prices = df_prices[mystocks]

In [22]:
df_prices.shape

(514, 706)

In [23]:
df_prices.head()

,2CRSI.PA,2MX.PA,ASP.PA,AALB.AS,ABI.BR,AB.PA,ABCA.PA,ABEO.PA,ABNX.PA,ABVX.PA,...,WLN.PA,XFAB.PA,XIL.PA,XIOR.BR,XPO.PA,YATRA.AS,CV.PA,ZENT.BR,^FCHI,^N100
Date,,,,,,,,,,,,,,,,,,,,,
2019-04-30,NaN,NaN,NaN,32.861336,NaN,NaN,5.799049,NaN,0.3585,NaN,...,NaN,NaN,NaN,NaN,283.446747,3.8,NaN,NaN,NaN,1086.579956
2019-05-02,9.90,NaN,3.905055,32.420052,72.227509,4.49,5.728651,30.643166,0.3790,10.42,...,55.750000,5.25,40.150002,38.143559,283.446747,3.8,NaN,15.400000,5538.859863,1078.500000
2019-05-03,9.82,NaN,3.871960,32.401276,72.300888,4.63,5.684651,29.654676,0.3790,10.12,...,54.400002,5.35,38.750000,38.189014,279.454529,3.8,NaN,15.400000,5548.839844,1080.239990
2019-05-06,9.76,NaN,3.867233,32.006943,72.044075,4.58,5.596654,30.939711,0.3500,10.46,...,54.750000,5.27,38.000000,38.279831,279.454529,3.8,NaN,15.500000,5483.520020,1069.140015
2019-05-07,9.48,NaN,3.763224,31.058657,70.371864,4.38,5.605453,30.148920,0.3435,10.18,...,52.049999,5.05,37.549999,39.051735,277.458435,3.8,NaN,16.200001,5395.750000,1052.900024


In [24]:
# Save df_prices to s3
_write_dataframe_to_csv_on_s3("euronext-stocks", df_prices, "df_prices.csv")

Writing 514 records to 'df_prices.csv' in bucket 'euronext-stocks'.


### Get ticker metadata

In [61]:
mytickers = yf.Tickers(mystocks)

In [62]:
# Get metadata (inspect an example for all the types of metadata that can be captured)
ticker_info = {}
for ticker in list(mytickers.tickers):
    print(ticker)
    ticker_dets = []
    info = mytickers.tickers[ticker].info
    try:
        ticker_dets.append(info["longName"])
    except:
        ticker_dets.append("Missing")
    try:
        ticker_dets.append(info["sector"])
    except:
        ticker_dets.append("Missing")
    try:
        ticker_dets.append(info["industry"])
    except:
        ticker_dets.append("Missing")
    ticker_info[ticker] = ticker_dets

2CRSI.PA
2MX.PA
ASP.PA
AALB.AS
ABI.BR
AB.PA
ABCA.PA
ABEO.PA
ABNX.PA
ABVX.PA
ABN.AS
ABO.BR
ACPH.BR
ACAN.PA
ACCEL.AS
ACCB.BR
AC.PA
ACNV.PA
AXS.AS
ACKB.BR
EOS.PA
ATI.PA
ALDV.PA
ALP.PA
ADOC.PA
ADP.PA
ADUX.PA
ADV.PA
ADVIC.PA
ADVI.PA
ADYEN.AS
AED.BR
AGN.AS
AKOM.PA
AGS.BR
AGFB.BR
AD.AS
AF.PA
AI.PA
AIR.PA
AJAX.AS
AKA.PA
AKW.PA
AKZA.AS
AAA.PA
ABIO.PA
ALD.PA
ALFEN.AS
CDANV.PA
CDA.PA
ALM.PA
ALO.PA
ALONV.PA
LTA.PA
ALTA.PA
AREIT.PA
ATE.PA
ALTUR.PA
ALX.AS
AMG.AS
AMPLI.PA
ACOMO.AS
AMUN.PA
APAM.AS
APM.PA
ARCAD.AS
MT.AS
ARG.PA
ARGX.BR
AKE.PA
ARTE.PA
PRC.PA
ARTO.PA
ASC.BR
ASIT.BR
ASM.AS
ASML.AS
ASRNL.AS
ASY.PA
ATA.PA
ATEME.PA
ATEB.BR
ATO.PA
ATRS.AS
AUB.PA
AUGR.PA
AURE.PA
AURS.PA
AVTX.AS
AVT.PA
CS.PA
CSNV.PA
AXW.PA
BSGR.AS
BCRA.PA
BAIN.PA
BALTA.BR
BALYO.PA
BAMNB.AS
BANI.BR
BUI.PA
BAR.BR
BFIT.AS
BASS.PA
BLC.PA
BESI.AS
BEFB.BR
BEKB.BR
FBEL.PA
BELR.BR
BELU.BR
BEN.PA
BBED.AS
BEVER.AS
BB.PA
BIG.PA
BCART.BR
BIM.PA
BLEE.PA
BNP.PA
BOI.PA
BOL.PA
BOLNV.PA
BON.PA
BOTHE.BR
BOOST.PA
BOKA.AS
BSD.PA
BGHL.AS
BGHS.AS
EN.

In [63]:
# Save ticker info to s3 (dataframe, then csv?)

s3 = boto3.client("s3")
serializedMyData = pickle.dumps(ticker_info)
s3.put_object(Bucket="euronext-stocks", Key="ticker_info", Body=serializedMyData)
# s3.put_object(Bucket="euronext-stocks", Key="ticker_info_test", Body=serializedMyData)

{'ResponseMetadata': {'RequestId': 'VQFVRZ52HWX2JGXD',
  'HostId': 'OXkM15O80E2/Ie+QhGQrEG7IuFHaTHidpMO7Fen1ZW0EIIlO/efQs/+DB2eOwVjLp2uYuhZp5z4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'OXkM15O80E2/Ie+QhGQrEG7IuFHaTHidpMO7Fen1ZW0EIIlO/efQs/+DB2eOwVjLp2uYuhZp5z4=',
   'x-amz-request-id': 'VQFVRZ52HWX2JGXD',
   'date': 'Thu, 22 Apr 2021 07:23:16 GMT',
   'etag': '"956db4a18bc4ff8e637b79e35177a825"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"956db4a18bc4ff8e637b79e35177a825"'}